In [ ]:
import requests
from bs4 import BeautifulSoup
import json


def scrape_bestbuy_ca_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract title
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else "N/A"

    # Extract short description from the meta tag
    description_tag = soup.find("meta", {"name": "description"})
    description = description_tag["content"] if description_tag else "N/A"

    # Extract embedded JSON from <script type="application/ld+json">
    specs = {}
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string.strip())
            if isinstance(data, dict) and data.get("@type") == "Product":
                specs = {
                    "brand": data.get("brand", {}).get("name", ""),
                    "sku": data.get("sku", ""),
                    "name": data.get("name", ""),
                    "category": data.get("category", ""),
                }
        except Exception:
            continue

    return {
        "title": title,
        "description": description,
        "specifications": specs
    }


# Example usage
url = "https://www.bestbuy.ca/en-ca/product/google-pixel-9a-128gb-obsidian-unlocked/18933640"
data = scrape_bestbuy_ca_product(url)
print(json.dumps(data, indent=2))

: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_bestbuy_ca_characteristics(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract product title
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else "N/A"

    # Extract meta description
    meta_desc = soup.find("meta", {"name": "description"})
    description = meta_desc["content"] if meta_desc else "N/A"

    # Extract specs from JSON inside script#__NEXT_DATA__
    script_tag = soup.find("script", {"id": "__NEXT_DATA__"})
    specs = {}
    if script_tag:
        try:
            json_data = json.loads(script_tag.string)
            # Navigate through the JSON structure to get characteristics
            product_info = json_data["props"]["pageProps"]["product"]
            specifications = product_info.get("details", {}).get("features", [])
            
            for spec_group in specifications:
                group_title = spec_group.get("title", "General")
                for feature in spec_group.get("features", []):
                    label = feature.get("featureName")
                    value = feature.get("featureValue")
                    if label and value:
                        specs[label] = value
        except Exception as e:
            print(f"Failed to parse specs: {e}")

    return {
        "title": title,
        "description": description,
        "specifications": specs
    }

# Example usage
url = "https://www.bestbuy.ca/en-ca/product/google-pixel-9a-128gb-obsidian-unlocked/18933640"
product_data = scrape_bestbuy_ca_characteristics(url)
print(json.dumps(product_data, indent=2))


: 